# Worksheet 08

Name:  Ketan Suhaas Saichandran
UID: U68176921

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.581042006786394, 5.037023018304715, 5.309208668390204, 5.666486114911809, 5.120617568874539, 3.3800387637019877, 4.131128689924663, 4.86508990708176, 4.733716039750574, 4.084389997555742]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.788416526169, 6.705609826368393, 9.23220040415905, 8.026891882779744, 7.535156071893178, 8.33216751771003, 8.685857069087149, 8.029615672989495, 9.635988190701353, 9.022487964708631]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1[-1]
        s1.pop()
        data.append(p1)
    else:
        p2 = s2[-1]
        s2.pop()
        data.append(p2)
print(data)

[9.022487964708631, 4.084389997555742, 9.635988190701353, 4.733716039750574, 8.029615672989495, 8.685857069087149, 4.86508990708176, 8.33216751771003, 4.131128689924663, 7.535156071893178]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. No of components = 2.
2. Means of components = [5, 8]
3. Standard deviations of components = [1, 1]
4. Weight of the components = [1/2, 1/2] (Uniform Distribution)



d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))
print(data)
s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

c:\Users\91960\anaconda3\envs\tf\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[9.022487964708631, 4.084389997555742, 9.635988190701353, 4.733716039750574, 8.029615672989495, 8.685857069087149, 4.86508990708176, 8.33216751771003, 4.131128689924663, 7.535156071893178]
[4.084389997555742, 4.733716039750574, 4.86508990708176, 4.131128689924663]
[9.022487964708631, 9.635988190701353, 8.029615672989495, 8.685857069087149, 8.33216751771003, 7.535156071893178]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.453581158578184,  mean_2 = 8.54021208118164
var_1 = 0.12202317741557435,  var_2 = 0.4614427644639257


- The mean of the distributions clustered are close to the true values.
- The variances are almost close to the true values.
- The weights of the distributions are close to the true values.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        #prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  9.022487964708631
probability of observing that point if it came from cluster 0 =  8.120647565294758e-38
probability of observing that point if it came from cluster 1 =  0.45645521340551354
point =  4.084389997555742
probability of observing that point if it came from cluster 0 =  0.6533283060185847
probability of observing that point if it came from cluster 1 =  2.66517767275059e-10
point =  9.635988190701353
probability of observing that point if it came from cluster 0 =  1.8344575920109756e-48
probability of observing that point if it came from cluster 1 =  0.15988581184682973
point =  4.733716039750574
probability of observing that point if it came from cluster 0 =  0.8280130033498937
probability of observing that point if it came from cluster 1 =  8.920235918618251e-08
point =  8.029615672989495
probability of observing that point if it came from cluster 0 =  1.9983381728673628e-23
probability of observing that point if it came from cluster 1 =  0.44275757387184544
point 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum( [x[0] * (x[1]-mean[0])**2 for x in zip(prob_s0_x,data)])/sum(prob_s0_x)  , sum( [x[0] * (x[1]-mean[1])**2 for x in zip(prob_s1_x,data)])/sum(prob_s1_x)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.3999999156717676,  P(S_2) = 0.6000000843282324
mean_1 = 4.453581077404535,  mean_2 = 8.54021156093353
var_1 = 0.12202317112990699,  var_2 = 0.4614446258605984


- The means of the distributions are close to the K means values.
- The variances are close to the K means values.
- The weights of the distributions are close to the K means values.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        #prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  9.022487964708631
probability of observing that point if it came from cluster 0 =  8.120587311973516e-38
probability of observing that point if it came from cluster 1 =  0.4564545086245036
point =  4.084389997555742
probability of observing that point if it came from cluster 0 =  0.6533284645053439
probability of observing that point if it came from cluster 1 =  2.665416985059907e-10
point =  9.635988190701353
probability of observing that point if it came from cluster 0 =  1.834440915661634e-48
probability of observing that point if it came from cluster 1 =  0.15988613096574497
point =  4.733716039750574
probability of observing that point if it came from cluster 0 =  0.8280128566568526
probability of observing that point if it came from cluster 1 =  8.920821163339606e-08
point =  8.029615672989495
probability of observing that point if it came from cluster 0 =  1.998328076556493e-23
probability of observing that point if it came from cluster 1 =  0.44275744027830133
point = 

There are very slight differences between the values of the probabilities, but the hard assignment will be the same in this case.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
hard_assignment = np.round(prob_s1_x)
for x in zip(hard_assignment,data):
    print(x[1]," belongs to ",x[0])

9.022487964708631  belongs to  1.0
4.084389997555742  belongs to  0.0
9.635988190701353  belongs to  1.0
4.733716039750574  belongs to  0.0
8.029615672989495  belongs to  1.0
8.685857069087149  belongs to  1.0
4.86508990708176  belongs to  0.0
8.33216751771003  belongs to  1.0
4.131128689924663  belongs to  0.0
7.535156071893178  belongs to  1.0


Even upon visual inspection, we could clearly say that the assignments to the distributions are correct.